In [2]:
# !pip install paramiko
# !pip install scp

In [1]:
import boto3
import numpy as np
import time
import datetime
import sys
from MLBD import MLBDApp
np.set_printoptions(threshold=sys.maxsize)

In [2]:
INSTANCE_SIZE = 1
client = boto3.client('ec2', region_name='us-east-1')
# Create SQS client
sqs = boto3.resource('sqs')
mlbd = MLBDApp()

In [20]:
# def send_mul(matrix_a = np.random.randint(10, size=(500, 500)),matrix_b = np.random.randint(10, size=(500, 500)),size_x=None, size_y=None):
#     """
#     Send the work to the workers
#     :param matrix_a: the first matrix
#     :param matrix_b: the second matrix
#     :param size_x: the size of the matrix
#     :param size_y: the size of the matrix
#     :return: the proof of work
#     """
#     # if size_x!=None: #if the size is given
#     #     matrix_a=np.random.randint(10, size=(size_x, size_y))
#     #     matrix_b=np.random.randint(10, size=(size_x, size_y))
#     #
#     # if matrix_b.shape!=matrix_a.shape: #if the matrix are not the same size
#     #     print("Shape of matrix are not the same")
#     #     return
#     #
#     # if len(matrix_b)<100: #if the matrix are too small
#     #     print("Maybe its not worth it to parallelize this work")
#     #     return
#
#     lenght=len(matrix_b)
#     # max_el=43000
#     max_el=100
#     split=max_el//lenght
#
#     if split>lenght:
#         split=lenght
#
#     rest=lenght%split
#     print("SENDING", lenght, split, rest)
#     nbr_msg=(len(matrix_a)//split)**2
#     msg=0
#
#     id=0
#     messages=[]
#     # Send the work to the queue
#     for i in range(int(len(matrix_a)/split)):
#
#         for j in range(int(len(matrix_b)/split)):
#
#             messages.append({
#                 'Id':str(i),
#                 'MessageBody':str({
#                     'A': matrix_a[i*split:i*split+split].tolist(),
#                     'B': matrix_b[0:,j*split:j*split+split].tolist(),
#                     'index': [i,j],
#                     'op':'*'
#                 })
#             })
#
#             # sqs.send_message(
#             #         QueueUrl=work_queue_url,
#             #         MessageBody=messages[0]['MessageBody']
#             # )
#
#             print("Messages 1", messages)
#
#             id=0
#             messages=[]
#             msg += 1
#             z = int(msg / nbr_msg * 25)
#             #print progress bar
#             sys.stdout.write('\r')
#             sys.stdout.write("[%-25s] %d%%" % ('=' * z, 4 * z))
#             sys.stdout.flush()
#
#         if rest>0:
#
#             messages.append({
#                 'Id':str(int(len(matrix_a)/split)),
#                 'MessageBody':str({
#                     'A': matrix_a[i*split:i*split+split].tolist(),
#                     'B': matrix_b[0:,lenght-rest:lenght].tolist(),
#                     'index': [i,lenght//split],
#                     'op':'*'
#                 })
#             })
#
#             print("Messages rest 1", messages)
#             # sqs.send_message(
#             #         QueueUrl=work_queue_url,
#             #         MessageBody=messages[0]['MessageBody']
#             # )
#             id=0
#             messages=[]
#
#     if rest>0:
#
#         for j in range(int(len(matrix_b)/split)):
#
#             messages.append({
#                 'Id':str(j),
#                 'MessageBody':str({
#                     'A': matrix_a[lenght-rest:lenght].tolist(),
#                     'B': matrix_b[0:,j*split:j*split+split].tolist(),
#                     'index': [lenght//split,j],
#                     'op':'*'
#                 })
#             })
#
#             # sqs.send_message(
#             #         QueueUrl=work_queue_url,
#             #         MessageBody=messages[0]['MessageBody']
#             # )
#             print("Messages rest 2", messages)
#             id=0
#             messages=[]
#
#         if rest>0:
#
#             messages.append({
#                 'Id':str(int(len(matrix_a)/split)),
#                 'MessageBody':str({
#                     'A': matrix_a[lenght-rest:lenght].tolist(),
#                     'B': matrix_b[0:,lenght-rest:lenght].tolist(),
#                     'index': [lenght//split,lenght//split],
#                     'op':'*'
#                 })
#             })
#
#             print("Messages rest 3", messages)
#
#             # sqs.send_message(
#             #         QueueUrl=work_queue_url,
#             #         MessageBody=messages[0]['MessageBody']
#             # )
#             id=0
#             messages=[]
#
#     return matrix_a,matrix_b,split,rest
#
#
# # In[54]:
#
#
# def receive_mul(proof,start):
#     """
#     Receive the results from the workers
#     :param proof: the proof of work (return [matrix_a,matrix_b,split,rest] of the send_mul function)
#     :param start: the time when the work was started
#     :return: the result
#     """
#     result = np.zeros(proof[0].shape,dtype=np.int64) #create the result matrix
#
#     nbr_msg=0
#     msg=0
#     if proof[3]>0:
#         nbr_msg+=(len(proof[0])//proof[2]+1)**2
#     else:
#         nbr_msg+=(len(proof[0])//proof[2])**2
#
#     print("\nRECEIVING")
#
#     while msg<nbr_msg:
#         # Receive message from SQS queue
#         response = queue.receive_messages(
#                 QueueUrl=results_queue_url,
#                 AttributeNames=['All'],
#                 MaxNumberOfMessages=10,
#                 VisibilityTimeout=30,
#                 WaitTimeSeconds=0
#             )
#
#         if len(response)>0:
#             for resp in response:
#                 dict =eval(resp.body)
#
#                 result[dict["index"][0]*proof[2]:dict["index"][0]*proof[2]+len(dict['value']),dict["index"][1]*proof[2]:dict["index"][1]*proof[2]+len(dict['value'][0])]=dict['value']
#
#                 sqs.delete_message(
#                         QueueUrl=results_queue_url,
#                         ReceiptHandle=resp.receipt_handle
#                     )
#                 msg+=1
#                 z=int(msg/nbr_msg*25)
#                 #print progress bar
#                 sys.stdout.write('\r')
#                 sys.stdout.write("[%-25s] %d%%" % ('=' * z, 4 * z))
#                 sys.stdout.flush()
#
#     print("\n****** RESULTS ******")
#     compute_time = time.time() - start
#     print("TIME : ", compute_time)
#
#     #Verify the result
#     if result.tolist() == np.dot(proof[0],proof[1]).tolist():
#         print("VERIFICATION : Okay")
#     else:
#         print("VERIFICATION : Not Okay")
#
#     calc_cost(compute_time,len(get_ip_worker()),nbr_msg)
#
#     #write matrix to file
#     with open('Mul_result.txt', 'w') as f:
#         for item in result.tolist():
#             f.write("%s " % item)
#
#     print("RESULT SAVED TO FILE")
#     print("****** RESULTS ******")
#
#     return result

In [6]:
# INSTANCE_SIZE = 1
# client = boto3.client('ec2', region_name='us-east-1')
# # Create SQS client
# sqs = boto3.resource('sqs')
#
# def get_default_security_group(client, key_name):
#     #extract key_name attribute from the security groups returned
#     response = [group[key_name] for group in client.describe_security_groups()['SecurityGroups'] if group['GroupName'] == 'default']
#
#     return response
#
# def get_key_pairs(client, removeExisting=False):
#     if removeExisting:
#         client.delete_key_pair(KeyName='airscholar-key')
#
#     keypairs = client.describe_key_pairs()['KeyPairs']
#     keypair = list(filter(lambda x: x['KeyName'] == 'airscholar-key', keypairs))
#
#     if not keypair:
#         keypair = client.create_key_pair(KeyName='airscholar-key')
#         f = io.StringIO(keypair['KeyMaterial'])
#         data = f.read()
#         file = open('labsuser.pem', 'w')
#         file.write(data)
#         file.close()
#     else:
#         keypair = keypair[0]
#
#     return keypair
#
# def launch_new_instance(client, keypair, count):
#     response = client.run_instances(
#         ImageId='ami-05723c3b9cf4bf4ff',
#         InstanceType='t2.micro',
#         KeyName=keypair,
#         MaxCount=count,
#         MinCount=count,
#         Monitoring={
#             'Enabled': True
#         },
#         SecurityGroupIds= get_default_security_group(client, key_name='GroupId')
#     )
#     ec2_inst_ids = [res["InstanceId"] for res in response]
#     waiter = client.get_waiter('instance_running')
#     waiter.wait(InstanceIds=[ec2_inst_ids])
#     return ec2_inst_ids
#
# def prepare_instances(client, keypair, count):
#     ec2 = boto3.resource('ec2')
#     ec2_inst_ids = []
#
#     deployed_count = 0
#     for instance in ec2.instances.all():
#         deployed_count += 1
#         client.start_instances(InstanceIds=[instance.id])
#         ec2_inst_ids.append(instance.id)
#
#     if deployed_count < count:
#         ec2_inst_ids.append(launch_new_instance(client, keypair, (count - deployed_count)))
#
#     if not ec2_inst_ids:
#         ec2_inst_ids.append(launch_new_instance(client, keypair, count))
#
#     return ec2, ec2_inst_ids
#
# def configure_ssh():
#     sshs = []
#     for count in range(0, INSTANCE_SIZE):
#         ssh = paramiko.SSHClient()
#         ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
#         sshs.append(ssh)
#     return sshs
#
# def ssh_connect_with_retry(ssh, ip_address, retries):
#     if retries > 3:
#         return False
#     f = open('labsuser.pem', 'r')
#     privkey = paramiko.RSAKey.from_private_key(f)
#     # print(privkey)
#     interval = 5
#     try:
#         retries += 1
#         print('SSH into the instance: {}'.format(ip_address))
#         ssh.connect(hostname=ip_address,
#                     username='ec2-user', pkey=privkey)
#         return True
#     except Exception as e:
#         print(e)
#         time.sleep(interval)
#         print('Retrying SSH connection to {}'.format(ip_address))
#         ssh_connect_with_retry(ssh, ip_address, retries)
#
# def ssh_disconnect(ssh):
#         """Close ssh connection."""
#         if ssh:
#             ssh.close()
#
# def get_public_address(ec2, instance_id):
#     # ec2 = boto3.resource('ec2', region_name='us-east-1')
#     instance = ec2.Instance(id=instance_id)
#     instance.wait_until_running()
#     current_instance = list(ec2.instances.filter(InstanceIds=[instance_id]))
#     ip_address = current_instance[0].public_ip_address
#     return ip_address
#
# def get_queue(sqs, queue_name='queue'):
#     # Get the queue. This returns an SQS.Queue instance
#     # There is no queue, create a new SQS queue
#     attributes = {
#         'DelaySeconds': '0',
#         'MessageRetentionPeriod': '86400',
#         "ReceiveMessageWaitTimeSeconds": "0"
#     }
#
#     for idx in range(INSTANCE_SIZE):
#         sqs.create_queue(
#             QueueName=f"{queue_name}{idx}",
#             Attributes=attributes
#         )
#
#         sqs.create_queue(
#             QueueName=f'result-queue-{idx}',
#             Attributes=attributes
#         )
#
# def send_message_to_queue(sqs, queue_name, message):
#     queue = sqs.get_queue_by_name(QueueName=queue_name)
#     # Send message to SQS queue
#     response = queue.send_messages(
#         Entries=message
#     )
#     # print(response)
#     return response
#
# def install_required_packages(ssh):
#     stdin, stdout, stderr = ssh.exec_command("sudo yum install pip -y && sudo pip install numpy boto3")
#     return stdout, stderr
#
# def initialise_instances(client):
#     sshs = configure_ssh()
#     keypair = get_key_pairs(client, False)
#     ec2, instances = prepare_instances(client, keypair['KeyName'], INSTANCE_SIZE)
#     ip_addresses = [get_public_address(ec2, instance) for instance in instances]
#     print(list(ip_addresses))
#     for idx in range(0, len(sshs)):
#         ssh = sshs[idx]
#         ip_address = ip_addresses[idx]
#         # connect to ssh
#         print(f"Conencting to Instance-{idx} with IP Address {ip_address}")
#         ssh_connect_with_retry(ssh, ip_address, 0)
#         # install required python packages
#         print(f"Installing required packages for Instance-{idx} with IP Address {ip_address}")
#         stdout, stderr = install_required_packages(ssh)
#         print(stdout.read().decode('utf-8'))
#         print(stderr.read().decode('utf-8'))
#
#         #configure aws access to the instance
#         print(f"Configuring Instance -{idx} with IP Address {ip_address} for remote access")
#         configure_aws_access_for_ssh(ssh, ip_address)
#
#         #upload worker file to the instance
#         scp = SCPClient(ssh.get_transport())
#         bulk_upload(scp, fetch_local_files('./worker'), '~', ip_address)
#
#         # start worker on the instance
#         print(f"Starting worker {idx}")
#         stdin, stdout, stderr = ssh.exec_command(f'python ./worker.py {idx}')
#         # # print(stdout.read().decode('utf-8'))
#         # # print(stderr.read().decode('utf-8'))
#
# def get_messages_from_queue(instance_size, queue, message_size=10):
#     messages = []
#     sqs = boto3.resource('sqs')
#     queue = sqs.get_queue_by_name(QueueName=queue)
#
#     for message in queue.receive_messages(MaxNumberOfMessages=message_size, MessageAttributeNames=['All'], WaitTimeSeconds=0):
#         messages.append(literal_eval(message.body))
#         message.delete()
#     return messages
#
# def split_row(array, nrows, ncols):
#     """
#     Return an array of shape (n, nrows, ncols) where
#     n * nrows * ncols = arr.size
#
#     If arr is a 2D array, the returned array should look like n subblocks with
#     each subblock preserving the "physical" layout of arr.
#     """
#     h, w = array.shape
#     assert h % nrows == 0, f"{h} rows is not evenly divisible by {nrows}"
#     assert w % ncols == 0, f"{w} cols is not evenly divisible by {ncols}"
#     return (array.reshape(h//nrows, nrows, -1, ncols)
#                .swapaxes(1,2)
#                .reshape(-1, nrows, ncols))
#
# def split_col(array, nrows, ncols):
#     """Split a matrix into sub-matrices."""
#     r, h = array.shape
#     return [np.vsplit(i, 5) for i in np.hsplit(arr1, r)]
#
# def generate_array(nrows, ncols):
#     arr = np.random.randint(5, size=(nrows, ncols))
#     # # print('arr 1:\n', arr)
#     # arr = split_row(arr, 1, split_size)
#     arr1 = np.random.randint(5, size=(nrows, ncols))
#     # print('arr 2:\n', arr1)
#     # arr1 = split_col(arr1, 1, split_size)
#
#     return arr, arr1
#
# def upload_file_to_s3(file_name, bucket, object_name=None):
#     """Upload a file to an S3 bucket
#
#     :param file_name: File to upload
#     :param bucket: Bucket to upload to
#     :param object_name: S3 object name. If not specified then file_name is used
#     :return: True if file was uploaded, else False
#     """
#
#     # If S3 object_name was not specified, use file_name
#     if object_name is None:
#         object_name = os.path.basename(file_name)
#
#     # Upload the file
#     s3_client = boto3.client('s3')
#     try:
#         response = s3_client.upload_file(file_name, bucket, object_name)
#     except:
#         # logging.error(e)
#         return False
#     return True
#
# def bulk_upload(scp, filepaths: list[str], remote_path, host):
#         """
#         Upload multiple files to a remote directory.
#
#         :param List[str] filepaths: List of local files to be uploaded.
#         """
#         try:
#             scp.put(
#                 filepaths,
#                 remote_path=remote_path,
#                 recursive=True
#             )
#             print(f"Finished uploading {len(filepaths)} files to {remote_path} on {host}")
#         except SCPException as e:
#             print(f"SCPException during bulk upload: {e}")
#         except Exception as e:
#             print(f"Unexpected exception during bulk upload: {e}")
#
# def configure_aws_access_for_ssh(ssh, ip_address):
#     """
#     This function extracts the AWS configuration you have locally and push to the server
#     :param ssh:ssh object
#     :return:
#     """
#     output = subprocess.getoutput("cat ~/.aws/credentials")
#     ssh.exec_command(f'mkdir ~/.aws && touch ~/.aws/credentials')
#     ssh.exec_command(f"echo '{output}' > ~/.aws/credentials")
#     print(f'SSH AWS configuration done for {ip_address}')
#
# def matrix_dot_product(matrix_a, matrix_b):
#     start_time = datetime.datetime.now()
#     result = []
#     for i in range(len(matrix_a)):
#         row = []
#         for j in range(len(matrix_b[0])):
#             sum = 0
#             for k in range(len(matrix_b)):
#                 sum += matrix_a[i][k] * matrix_b[k][j]
#             row.append(sum)
#         result.append(row)
#     print('Computation time', datetime.datetime.now() - start_time)
#
#     return result
#
# def matrix_add(matrix_1, matrix_2):
#     start_time = datetime.datetime.now()
#     result = []
#     for idx_row in range(0, len(matrix_1)):
#         row = matrix_1[idx_row]
#         row1 = matrix_2[idx_row]
#         cols = []
#         for idx_col in range(0, len(row)):
#             cols.append(row[idx_col] + row1[idx_col])
#         result.append(cols)
#     print('Computation time', datetime.datetime.now() - start_time)
#     return result
#
# def delete_queues(sqs):
#     #get all the queues
#     client = boto3.client('sqs')
#     response = client.list_queues(
#     MaxResults=123)
#
#     for queue in response['QueueUrls']:
#         client.delete_queue(QueueUrl= queue)
#         print(f'{queue} deleted!')
#
# def terminate_instances():
#     ec2 = boto3.resource('ec2')
#     for instance in ec2.instances.all():
#         # terminate all instances
#         print(f'Terminating {instance.id}')
#         instance.terminate()
#
# terminate_instances()
# mlbd.delete_queues(mlbd.sqs)

Terminating i-042276f6fa0ee39a8
Terminating i-00f0ed952f24e950d


KeyError: 'QueueUrls'

In [5]:
mlbd.get_queue(sqs)

ClientError: An error occurred (ExpiredToken) when calling the CreateQueue operation: The security token included in the request is expired

In [6]:
mlbd.initialise_instances(client)

ClientError: An error occurred (RequestExpired) when calling the DescribeKeyPairs operation: Request has expired.

### ADDITION

In [ ]:
   # [print(idx, dt) for idx in range(min(dt), max(dt)+1)]
        # result = []
        # for i in range(min(dt), max(dt)+1):
        #     row = []
        #     start = min(dt)
        #     end = max(dt)+1
        #     for j in range(start, end):
        #         idx = []
        #         row_sum = sum(arr[i]*arr1.T[j])
        #         row.append(row_sum)
        #     print(row)
        #     # print(id, '=>', start, end)
        # result.append(row)
# np.array(result)

In [227]:
# task1(15, 5, 'addition')

In [25]:
def split_and_compute_dot_product(matrix_a, matrix_b, chunk_size):
    """
    Splits two huge matrices into smaller chunks and computes the dot product for each pair of chunks without using any libraries.

    Parameters:
    - matrix_a (list): The first matrix to split and compute the dot product for.
    - matrix_b (list): The second matrix to split and compute the dot product for.
    - chunk_size (int): The size of each chunk.

    Returns:
    - dot_products (list): A list of dot products for each pair of chunks.
    """
    # Split the matrices into chunks
    chunks_a = [matrix_a[i:i+chunk_size] for i in range(0, len(matrix_a), chunk_size)]
    chunks_b = [matrix_b[i:i+chunk_size] for i in range(0, len(matrix_b), chunk_size)]

    # Initialize a list to store the dot products
    dot_products = []

    # Iterate over the chunks and compute the dot product for each pair of chunks
    for chunk_a, chunk_b in zip(chunks_a, chunks_b):
        print(chunk_a, chunk_b)
        dot_product = [[0 for j in range(len(chunk_a))] for i in range(len(chunk_b[0]))]
        print(dot_product)
        for i in range(len(chunk_b)):
            for j in range(len(chunk_a[0])):
                for k in range(len(chunk_b[0])):
                    dot_product[i][j] += chunk_b[i][k] * chunk_a[k][j]

        dot_products.append(dot_product)

    return dot_products

In [26]:
split_and_compute_dot_product(arr, arr1, 2)

[[3 1 3 3]
 [1 1 2 4]] [[4 0 0 4]
 [1 1 0 4]]
[[0, 0], [0, 0], [0, 0], [0, 0]]


IndexError: index 2 is out of bounds for axis 0 with size 2

In [22]:
arr1

array([[2, 4, 0, 3, 3, 3, 0, 3, 4, 4],
       [0, 1, 2, 0, 3, 0, 1, 0, 3, 1],
       [3, 4, 1, 1, 1, 0, 2, 1, 1, 0],
       [0, 4, 2, 3, 4, 0, 0, 4, 1, 4],
       [0, 2, 0, 3, 1, 0, 3, 0, 0, 2],
       [1, 1, 1, 4, 2, 3, 4, 2, 4, 3],
       [4, 2, 4, 3, 0, 1, 2, 4, 3, 0],
       [1, 2, 3, 4, 1, 4, 2, 1, 3, 3],
       [0, 4, 3, 1, 3, 2, 2, 2, 2, 1],
       [3, 4, 1, 0, 3, 3, 2, 1, 1, 0]])

In [23]:
array_size = 4
chunk_size = 2
operation = 'multiply'
# print(f'Generating {array_size}x{array_size} matrix')
arr, arr1 = generate_array(array_size,array_size)
# print(f'Splitting {array_size}x{array_size} matrix into {chunk_size}x{chunk_size}')
# s_arr = split_row(arr, chunk_size, chunk_size)
# s_arr1 = split_row(arr1.T, chunk_size, chunk_size)
# print(arr1.T, '\n')
# print(s_arr1)
#manual matrix addition
# print(f'Computing manual addition of {array_size}x{array_size} matrix')
# res1 = np.array(matrix_dot_product(arr, arr1))
# print(res1)
# # print(f"\nComputing distribution {operation} operation for {array_size}x{array_size}")
# compute_matrix_operation(operation, s_arr, s_arr1)
# #wait for 15 seconds
# print('Wait 60 seconds before computing result')
# time.sleep(60)
# #merge all the queue results
# print(f"\nMerging computation results")
# res2 = merge_queue_result(s_arr, array_size, chunk_size)
# print(f"\nVerifying computation results")
# print(np.array(res1 == res2))

In [19]:
#create instance
#configure instance
#create matrix
#split matrix
#send matrix to the queue
#read matrix from the queue on the instance created
#compute matrix
#send result to base


In [145]:
!git add . && git commit -am "Updated Addition Workflow"
!git push

[main 356fbbd] Updated Addition Workflow
 1 file changed, 9 insertions(+), 10 deletions(-)
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 8 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 469 bytes | 469.00 KiB/s, done.
Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/airscholar/MLCloudComputing-python.git
   2d62b4f..356fbbd  main -> main
